In [1]:
import menpo.io as mio
import os, sys
from menpofit.aam import HolisticAAM, LucasKanadeAAMFitter, PatchAAM
from menpo.feature import fast_dsift, double_igo
from menpo.visualize import print_progress
from menpo.landmark import labeller, face_ibug_68_to_face_ibug_68_trimesh
from menpofit.fitter import noisy_shape_from_bounding_box

#file path
train_dir =  './trainset/'
saveFinalDir = './paam/'
data_dir = './t/'

#landmark resolver takes image path
def find_landmark(asset):
    index = str(asset).rfind('/') + 1
    anotation_dir + str(asset)[ index: -4 ]
    return { "MyAAM": os.path.abspath( anotation_dir + '/' + str(asset)[ index: -4 ] + '.pts' ) }

#lazy import. copy from menpo
def load_dataTrain(path_to_images, crop_percentage, max_images=None):
    images = []
    # load landmarked images
    for i in print_progress(mio.import_images(path_to_images, max_images=max_images, verbose=True)):
        # crop image
        i = i.crop_to_landmarks_proportion(crop_percentage)
        i = i.resize((804,688))
        # convert it to grayscale if needed
        if i.n_channels == 3:
            i = i.as_greyscale()
        # append it to the list
        #labeller(i, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
        images.append(i)
    return images

def load_dataPredict(path_to_images,max_images=None):
    images = []
    # load landmarked images
    for i in print_progress(mio.import_images(path_to_images,max_images=max_images, verbose=True)):
        images.append(i)
    return images


training_images = load_dataTrain(train_dir, 0.2)


# aam = HolisticAAM(training_images, group='face_ibug_68_trimesh', diagonal=150, 
#     scales=(0.5, 1.0), holistic_features=fast_dsift, verbose=True,
#                         max_shape_components=20, max_appearance_components=150)
aam = PatchAAM(training_images, group='PTS', patch_shape=[(15, 15), (23, 23)],
                     diagonal=150, scales=(0.5, 1.0), holistic_features=fast_dsift,
                     max_shape_components=20, max_appearance_components=200,
                     verbose=True)

mio.export_pickle(aam, './aammtrained.pkl', overwrite=False, protocol=2)
fitter = LucasKanadeAAMFitter(aam,n_shape=[5, 20], 
                              n_appearance=[30, 150])



Found 2000 assets, index the returned LazyList to import.
[====================] 100% (2000/2000) - done.                                 
- Computing reference shape                                                     Computing batch 0
- Building models
  - Scale 0: Done
  - Scale 1: Done
                                                              

In [2]:
%matplotlib inline
files = [ name for name in os.listdir( data_dir ) if name.endswith('.png') ]
# ErrorFile = []
for file in print_progress(files):
    fileName = file[:-4]
    img = mio.import_image(data_dir + file )
    img.landmarks['BBox'] = img.landmarks['PTS'].lms.bounding_box()
    #s = noisy_shape_from_bounding_box(gt_s, gt_s.bounding_box())
    try:
    #result = fitter.fit_from_shape(img,  img.landmarks['BBox'].lms,max_iters=[15, 5])
    result = fitter.fit_from_bb(img, img.landmarks['BBox'].lms, max_iters=[15, 5])
    #result.view(render_initial_shape=True)
    #lmks = result.final_shape.as_pointgraph()
    #lmks.landmarks['PTS'] = result.final_shape
    #mio.export_landmark_file(lmks.landmarks['PTS'], saveFinalDir + fileName +'_kpt.pts', overwrite=True)
    lmks = result.final_shape
    lmks.landmarks['PTS'] = lmks
    mio.export_landmark_file(lmks.landmarks['PTS'], saveFinalDir + fileName +'_kpt.pts', overwrite=True)
    except:
        pass


   
    #mio.export_landmark_file(result.final_shape.landmarks['PTS'], saveFinalDir + fileName +'_kpt.pts', overwrite=True)

IndentationError: expected an indented block (<ipython-input-2-f3ea8dfc5d63>, line 11)

In [ ]:
from menpowidgets import visualize_images
from menpodetect import load_dlib_frontal_face_detector
# # Load detector
from menpofit.aam import WibergInverseCompositional

fitter = LucasKanadeAAMFitter(aam, n_shape=[5, 20], n_appearance=[30, 150])


detect = load_dlib_frontal_face_detector()

a = mio.import_image("./t/127627778_1.jpg")
a = a.as_greyscale()

a.landmarks['BBox'] = a.landmarks['PTS'].lms.bounding_box()
result = fitter.fit_from_bb(a, a.landmarks['BBox'].lms,max_iters=50)
result.view_widget()